In [2]:
from insurence_premium.entity.model_entity import ModelEvaluationArtifact

In [3]:
import os
os.getcwd()
os.chdir("../")

In [4]:
from dataclasses import dataclass
@dataclass
class DataModelPusher:
    model_pusher_file_path: str

    

In [67]:
from insurence_premium.constant import *
from insurence_premium.util.common import (
    
    create_directories,read_yaml,
    load_numpy_array_data,load_data,load_object,write_yaml_file
)
from pathlib import Path
from insurence_premium.entity import (
    DataIngestionConfig,
    DataValidationConfig,
    DataTransformationConfig,
    DataModelTrainerConfig
)
from insurence_premium import logger


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.time_stamp=CURRENT_TIME_STAMP

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(
        self,
    ) -> DataIngestionConfig:
        ingestion_config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(ingestion_config.root_dir),
            raw_data=Path(ingestion_config.raw_data),
            ingested_train_dir=Path(ingestion_config.ingested_train_dir),
            ingested_test_dir=Path(ingestion_config.ingested_test_dir),
        )
        return data_ingestion_config

    def get_data_validation_config(self) -> DataValidationConfig:
        validation_config = self.config.data_validation

        data_validation_config = DataValidationConfig(
            root_dir=Path(validation_config.root_dir),
            schema_file_path=Path(validation_config.schema_file_path),
            report_file_path=Path(validation_config.report_file_path),
            report_page_file_path=Path(validation_config.report_page_file_path),
        )
        return data_validation_config

    def get_data_transformation_config(self) -> DataTransformationConfig:
        try:
            transformation_config = self.config.data_transformation
            data_transformation_config = DataTransformationConfig(
                root_dir=Path(transformation_config.root_dir),
                transformed_train_file_path=Path(
                    transformation_config.transformed_train_file_path
                ),
                transformed_test_file_path=Path(
                    transformation_config.transformed_test_file_path
                ),
                preprocessing_dir=Path(transformation_config.preprocessing_dir),
                preprocessing_file_path=Path(
                    transformation_config.preprocessing_file_path
                ),
            )
            return data_transformation_config
            logging.info(f"return: [{data_ingestion_config}]")
        except Exception as e:
            raise e

    def get_model_trainer_config(self) -> DataModelTrainerConfig:
        try:
            model_trainer_config = self.config.data_model_trainer

            data_model_trainer_config = DataModelTrainerConfig(
                root_dir=Path(model_trainer_config.root_dir),
                trained_model_file_path=
                    model_trainer_config.trained_model_file_path
                ,
                base_accuracy=model_trainer_config.base_accuracy,
                model_config_file_path=Path(
                    model_trainer_config.model_config_file_path
                ),
            )
            return data_model_trainer_config
            logger.info(f"model trainer config : {data_model_trainer_config}")
        except Exception as e:
            raise e

    def get_model_evaluation_config(self):
        try:
            data_evaluation_config=self.config.data_model_evaluation
            model_evaluation_file_path=os.path.join(data_evaluation_config.root_dir ,data_evaluation_config.model_evaluation_file_name)
            model_evaluation_config=DataModelEvaluation(
                root_dir= Path(data_evaluation_config.root_dir),
                model_evaluation_file_path= Path(model_evaluation_file_path),
                time_stamp= self.time_stamp

                
            )
            return model_evaluation_config
        except Exception as e:
            raise e
    def get_model_pusher_config(self):
        try:
            model_pusher_config=self.config.model_pusher

            data_model_pusher=DataModelPusher(model_pusher_file_path=model_pusher_config.root_dir)


            return data_model_pusher
            logger.info(f"model_pusher config:{data_model_pusher}")

            
        except Exception as e:
            raise e        

In [68]:
ConfigurationManager().get_model_pusher_config()

[2023-04-12 22:22:50,645: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-12 22:22:50,652: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-12 22:22:50,656: INFO: common]: created directory at: artifacts


DataModelPusher(model_pusher_file_path='prediction_services/saved_model/model.pkl')

In [69]:
from insurence_premium import logger
from insurence_premium.entity.model_entity import ModelEvaluationArtifact 

import os, sys
import shutil


class ModelPusher:

    def __init__(self, config:ConfigurationManager):
        try:
            logger.info(f"{'>>' * 30}Model Pusher log started.{'<<' * 30} ")
            self.config= ConfigurationManager()
            self.model_pusher_config=self.config.get_model_pusher_config()
            self.model_evaluation_config=self.config.get_model_evaluation_config()

        except Exception as e:
            raise e

    def export_model(self):
        try:
            evaluated_model_file_path = str(self.model_evaluation_config.model_evaluation_file_path)
            logger.info(f"{evaluated_model_file_path}")
            export_model_file_path = self.model_pusher_config.model_pusher_file_path

            export_dir=os.path.dirname(export_model_file_path)
            logger.info(f"Exporting model file: [{export_model_file_path}]")
            os.makedirs(export_dir, exist_ok=True)

            shutil.copy(src=evaluated_model_file_path, dst=export_model_file_path)
            #we can call a function to save model to Azure blob storage/ google cloud strorage / s3 bucket
            logger.info(
                f"Trained model: {evaluated_model_file_path} is copied in export dir:[{export_model_file_path}]")

            
            
            return export_model_file_path
        except Exception as e:
            raise e

    

    def __del__(self):
        logger.info(f"{'>>' * 20}Model Pusher log completed.{'<<' * 20} ")

In [70]:
con= ConfigurationManager()
model_pusher=ModelPusher(config=con)
model_pusher.export_model()

[2023-04-12 22:22:59,307: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-12 22:22:59,312: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-12 22:22:59,318: INFO: common]: created directory at: artifacts
[2023-04-12 22:22:59,322: INFO: 735480471]: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Model Pusher log started.<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
[2023-04-12 22:22:59,332: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-12 22:22:59,344: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-12 22:22:59,349: INFO: common]: created directory at: artifacts
[2023-04-12 22:22:59,354: INFO: 735480471]: >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Model Pusher log completed.<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
[2023-04-12 22:22:59,357: INFO: 735480471]: artifacts\data_model_evaluation\model_e.yaml
[2023-04-12 22:22:59,360: INFO: 735480471]: Exporting model file: [pred

'prediction_services/saved_model/model.pkl'

In [55]:
a=str(ConfigurationManager().get_model_evaluation_config().model_evaluation_file_path)
a

[2023-04-12 21:06:34,273: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-04-12 21:06:34,280: INFO: common]: yaml file: params.yaml loaded successfully
[2023-04-12 21:06:34,285: INFO: common]: created directory at: artifacts


'artifacts\\data_model_evaluation\\model_e.yaml'

In [26]:
from insurence_premium.entity.config_entity import DataModelEvaluation

In [66]:
from insurence_premium.config import ConfigurationManager